# Scraping: récupération de la page d'accueil de la RTBF

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [La RTBF](https://www.rtbf.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [13]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de tous les articles de la page d'accueil



In [14]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles = []

root_url = f"https://www.rtbf.be"
response = requests.get(root_url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
for link in soup.find_all("a", {"class":"stretched-link outline-hidden"}):
    title = link.text.strip()
    url = root_url + link.get('href')
    articles.append([url, title])

In [15]:
# Affichage du nombre d'articles récupérés
len(articles)

80

In [16]:
# Affichage des 10 premières entrées
articles[:10]

[['https://www.rtbf.be/article/mondiaux-de-gymnastique-que-peut-on-attendre-de-la-belgique-sans-nina-derwael-11616221',
  'Mondiaux de gymnastique\xa0: que peut-on attendre de la Belgique sans Nina Derwael\xa0?'],
 ['https://www.rtbf.be/article/l-actu-culturelle-de-la-semaine-diane-keaton-les-chaines-musicales-de-mtv-et-la-mediatheque-nouvelle-11617983',
  'L’actu culturelle de la semaine\xa0: Diane Keaton, les chaînes musicales de MTV et la Médiathèque Nouvelle'],
 ['https://www.rtbf.be/article/50-plus-efficace-la-nouvelle-generation-de-panneaux-photovoltaiques-est-en-preparation-11599613',
  '50% plus efficace, la nouvelle génération de panneaux photovoltaïques est en préparation'],
 ['https://www.rtbf.be/article/une-entree-en-matiere-difficile-de-bon-augure-pour-la-suite-david-hubert-revient-sur-ses-debuts-avec-l-union-11618639',
  '"Une entrée en matière difficile", "de bon augure pour la suite"\xa0: David Hubert revient sur ses débuts avec l’Union'],
 ['https://www.rtbf.be/article

## Création d'un dataframe avec les liens et les titres des articles


In [17]:
df = pd.DataFrame(articles, columns=['link', 'title'])
df

,link,title
0,https://www.rtbf.be/article/mondiaux-de-gymnas...,Mondiaux de gymnastique : que peut-on attendre...
1,https://www.rtbf.be/article/l-actu-culturelle-...,L’actu culturelle de la semaine : Diane Keaton...
2,https://www.rtbf.be/article/50-plus-efficace-l...,"50% plus efficace, la nouvelle génération de p..."
3,https://www.rtbf.be/article/une-entree-en-mati...,"""Une entrée en matière difficile"", ""de bon aug..."
4,https://www.rtbf.be/article/quiz-sur-ces-films...,Quiz sur ces films et séries qu’on adore regar...
...,...,...
75,https://www.rtbf.behttps://auvio.rtbf.be/media...,Un monde meilleur
76,https://www.rtbf.behttps://auvio.rtbf.be/media...,Une brique dans le ventre
77,https://www.rtbf.behttps://auvio.rtbf.be/media...,Météo 20h00
78,https://www.rtbf.behttps://auvio.rtbf.be/media...,Le 19h30


In [18]:
# Sauvegarde du dataframe dans un fichier csv
df.to_csv(f"../data/rtbf_{time.strftime('%Y%m%d')}.csv", index=False)

OSError: Cannot save file into a non-existent directory: '..\data'

## Téléchargement d'un article et affichage du texte

In [19]:
# Récupération du premier lien du dataframe
article_url = df.iloc[0][0]

# Ouverture de la page
response = requests.get(article_url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
# Parsing du contenu
title = soup.find("h1").get_text(separator=' ')

paragraphs = []
for paragraph in soup.find_all("p", attrs={"class": None}):
    paragraphs.append(paragraph.get_text(separator=' ').strip())

content = "\n".join(paragraphs)
print(title)
print("==================================")
print(content)

Mondiaux de gymnastique : que peut-on attendre de la Belgique sans Nina Derwael ?
Six gymnastes belges composent la délégation belge pour les championnats du monde de gymnastique artistique du 19 au 25 octobre à Jakarta en Indonésie. C’est le premier rendez-vous sans Nina Derwael qui a donc mis un terme à sa carrière cet été.
Les gymnastes évolueront individuellement, avec pour objectif de décrocher une place en finale du concours général ou en finales par agrès, chaque nation ne pouvant aligner que six gymnastes chez les messieurs et quatre chez les dames, au maximum, avec aussi un maximum de trois représentants par agrès.
Lisa Vaelen  (De Gympies Keerbergen) et  Jade Vansteenkiste  (Izegem) seront les porte-drapeaux de la Belgique côté féminin.
C’est la deuxième fois sa carrière que Lisa Vaelen participera aux mondiaux. L’Anversoise, 21 ans, avait fait ses débuts sur la scène mondiale il y a trois ans à Liverpool, en Angleterre, terminant sixième en finale du saut et 11e du concours 

### Nettoyage du texte à l'aide d'expressions régulières

In [20]:
# Suppression de tout ce qui se trouve entre parenthèses
clean_content = re.sub("\([^\)]+\)", " ", content)
# Suppression des espaces multiples
clean_content = re.sub("\s+", " ", clean_content)

print(content)
print("==================================")
print(clean_content)


Six gymnastes belges composent la délégation belge pour les championnats du monde de gymnastique artistique du 19 au 25 octobre à Jakarta en Indonésie. C’est le premier rendez-vous sans Nina Derwael qui a donc mis un terme à sa carrière cet été.
Les gymnastes évolueront individuellement, avec pour objectif de décrocher une place en finale du concours général ou en finales par agrès, chaque nation ne pouvant aligner que six gymnastes chez les messieurs et quatre chez les dames, au maximum, avec aussi un maximum de trois représentants par agrès.
Lisa Vaelen  (De Gympies Keerbergen) et  Jade Vansteenkiste  (Izegem) seront les porte-drapeaux de la Belgique côté féminin.
C’est la deuxième fois sa carrière que Lisa Vaelen participera aux mondiaux. L’Anversoise, 21 ans, avait fait ses débuts sur la scène mondiale il y a trois ans à Liverpool, en Angleterre, terminant sixième en finale du saut et 11e du concours général.
Son objectif sera encore de rejoindre une finale, elle qui s’alignera au 

### Création d'un fichier avec le contenu de l'article


In [21]:
with open("../data/rtbf_example.txt", "w") as writer:
    writer.write(f"{title}\n\n{clean_content}")

FileNotFoundError: [Errno 2] No such file or directory: '../data/rtbf_example.txt'

## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp

## Télécharger automatiquement des PDF

In [1]:
request.get("https://max.de.wilde.web.ulb.be/camille/",verify=False)

NameError: name 'request' is not defined